In [2]:
import torch
from torch import tensor
import numpy as np
from tqdm import tqdm
import torch.utils.data as Data
from torch.nn import init
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
mnist_train = torchvision.datasets.FashionMNIST(
    root='~/Datasets/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.ToTensor())  # 将所有数据转换为Tensor
mnist_test = torchvision.datasets.FashionMNIST(
    root='~/Datasets/FashionMNIST',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
# 2、通过Dataloader读取小批量数据样本
batch_size = 256
train_iter = torch.utils.data.DataLoader(
    mnist_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)
test_iter = torch.utils.data.DataLoader(
    mnist_train,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

num_inputs = 784
num_outputs = 10  # 共10类
class softmaxnet(torch.nn.Module):
    def __init__(self, n_features, n_labels):
        super(softmaxnet, self).__init__()
        self.linear = torch.nn.Linear(n_features, n_labels)
    def forward(self, x):
        x_ = x.view((-1, num_inputs))
        y_ = self.linear(x_)
        return y_
    
net = softmaxnet(num_inputs, num_outputs)
lr = 0.1
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)

def get_test_info(data_iter, net):
    right_count, all_count = 0.0, 0
    for x, y in data_iter:
        y_ = net(x)
        l = loss(y_, y)
        right_count += (y_.argmax(dim=1)==y).sum().item()
        all_count += y.shape[0]
    return right_count/all_count, l.item()
num_epoch = 5


for epoch in range(num_epoch):
    train_r_num, train_all_num = 0.0, 0
    for X, y in tqdm(train_iter):
        y_ = net(X)
        l = loss(y_, y)
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_r_num += (y_.argmax(dim=1) == y).sum().item()
        train_all_num += y.shape[0]
    test_acc, test_ave_loss = get_test_info(test_iter, net)
    print('epoch %d, train loss %.4f, train acc %.3f' % (epoch+1, l.item(), train_r_num/train_all_num))
    print('test loss %.4f, test acc %.3f' % (test_ave_loss, test_acc))


  0%|                                                                                          | 0/235 [00:00<?, ?it/s]

epoch 1, train loss 0.6313, train acc 0.747
test loss 0.5513, test acc 0.792


  0%|                                                                                          | 0/235 [00:00<?, ?it/s]

epoch 2, train loss 0.4444, train acc 0.811
test loss 0.4733, test acc 0.822


  1%|█                                                                                 | 3/235 [00:00<00:11, 19.79it/s]

epoch 3, train loss 0.5256, train acc 0.825
test loss 0.4603, test acc 0.829


  0%|▎                                                                                 | 1/235 [00:00<00:23,  9.83it/s]

epoch 4, train loss 0.4397, train acc 0.831
test loss 0.4781, test acc 0.822


100%|████████████████████████████████████████████████████████████████████████████████| 235/235 [00:22<00:00, 10.40it/s]


epoch 5, train loss 0.4250, train acc 0.836
test loss 0.4499, test acc 0.834
